## Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
from faker import Faker
import psycopg2
from psycopg2.extensions import AsIs
import seaborn as sns
import matplotlib.pyplot as plt


## Funções

In [ ]:
def open_data_execel(data):
    # Abrindo planilhas
    df1 = pd.read_excel(
        data, sheet_name='Year 2009-2010'
    )
    df2 = pd.read_excel(
        data, sheet_name='Year 2010-2011'
    )

    # Concatenando planilhas
    df_concat = pd.concat([df1, df2])

    return df_concat


In [ ]:
def negative_values(data):
    result_menor_zero = data[['Quantity', 'Price']].query(
        "Quantity < 0 or Price < 0.0")

    result_zero = data[['Quantity', 'Price']].query(
        "Quantity == 0 or Price == 0.0")

    print(f'Total valores negativos: {len(result_menor_zero)}')
    print(f'Total Valores zerados: {len(result_zero)}')

    return None


In [ ]:
def cleaning_transformation_data(data):
    # Removendo nulos
    data = data.dropna()

    # Removendo 0
    data = data.copy()
    data = data[(data['Quantity'] > 0)]
    data = data.replace(0.0, pd.NA).dropna()

    # Removendo inconcistencia
    conditions = {
        'Country': ['European Community', 'Unspecified'],
        'Description': ['This is a test product.', 'Manual', 'Discount', 'Next Day Carriage', 'High Resolution Image'],
        'StockCode': ['BANK CHARGES', 'C2', 'DOT', 'PADS', 'POST', 'SP1002', 'ADJUST', 'ADJUST2']
    }

    data = data[~data['Country'].isin(conditions['Country'])]
    data = data[~data['Description'].isin(conditions['Description'])]
    data = data[~data['StockCode'].isin(conditions['StockCode'])]

    # Padronizando nome paises
    data.loc[data['Country'] == 'RSA', 'Country'] = 'South Africa'

    # Coluna total vendido
    data['TotalValue'] = np.multiply(data['Quantity'], data['Price'])

    # Coluna id vendas
    sales = []
    for i in data['Invoice']:
        sale = "INV" + str(i)
        sales.append(sale)

    data['SalesID'] = sales

    # Renomeando colunas
    data = data.rename(columns={
        'StockCode': 'ProductID',
        'Description': 'Product',
        'InvoiceDate': 'Date',
        'Customer ID': 'ClientID'
    })

    dataset = data

    return dataset


In [ ]:
def create_table_client(data):
    # Extraindo colunas
    client = data.copy()
    client = client[['ClientID', 'Country']]

    # Removendo duplicados
    client.drop_duplicates(
        subset=['ClientID'],
        keep='first',
        inplace=True
    )

    # Convertendo para inteiro
    client['ClientID'] = client['ClientID'].astype(int)

    # Ordenando pela coluna de id
    client = client.sort_values(
        by='ClientID',
        ascending=True
    )

    # Nomeando clientes
    # Definindo uma semente para a função Faker
    Faker.seed(10)
    faker = Faker()

    # Criando lista de nomes
    nomes_empresas = []
    while len(nomes_empresas) < len(client):
        nome_empresa = faker.company()
        if nome_empresa not in nomes_empresas:
            nomes_empresas.append(nome_empresa)

    # Criando emails para os clientes
    emails = []
    for name in nomes_empresas:
        email = name.replace(",", "").lower() + "@email.com"
        email = email.replace(" ", "")
        emails.append(email)

    # Inseridno novos dados e ordenando o dataframe
    client['ClientName'] = nomes_empresas
    client['Email'] = emails
    order = ['ClientID', 'ClientName', 'Email', 'Country']
    client = client.reindex(columns=order)

    # Transformando em lista
    client.tolist()

    return client


In [ ]:
def create_table_sales(data):
    # Criando o dataframe de Vendas
    sales = data.copy()
    sales = sales[['SalesID', 'ClientID', 'Invoice',
                   'Date', 'TotalValue']]

    # Removendo horas e convertendo data
    sales['Date'] = sales['Date'].dt.date
    sales['Date'] = pd.to_datetime(sales['Date'])

    # Agrupando por total de vendas
    sales = sales.groupby(['SalesID', 'ClientID',
                           'Invoice', 'Date'])[
        'TotalValue'].sum().reset_index()

    # Ordenando
    sales = sales.sort_values('Invoice', ascending=True)

    # Transformando em lista
    sales.tolist()

    return sales


In [ ]:
def create_table_product(data):
    # Criando dataframe para os Produtos
    product = data.copy()
    product = product[['ProductID', 'Product', 'Price']]

    # Removendo duplicados
    product.drop_duplicates(subset=['ProductID'],
                            keep='first',
                            inplace=True
                            )

    # Redefinindo a coluna de id e ordenando
    product['ProductID'] = product['ProductID'].astype(str)
    product = product.sort_values('ProductID', ascending=True)

    # Transformando em lista
    product.tolist()

    return product


In [ ]:
def create_table_salesitems(data):
    # Criando dataframe itens vendidos
    salesitems = data.copy()
    salesitems = salesitems[['SalesID', 'ProductID', 'Invoice',
                            'Quantity', 'Price', 'TotalValue']]

    # Removendo produtos duplicados
    salesitems = salesitems.groupby(['SalesID', 'ProductID']).agg({
        'Invoice':    'first',
        'Quantity':   'sum',
        'Price':      'first',
        'TotalValue': 'sum'
    }).reset_index()

    # Transformando em lista
    salesitems.tolist()

    return salesitems


In [ ]:
def connect_db_postgres(database):
    # Conectando ao SGBD
    conn = psycopg2.connect(host='localhost',
                            database=database,
                            user='postgres',
                            password='123456')

    # Commit automatico
    conn.autocommit = True

    # Criando cursor para comandos SQL
    cursor = conn.cursor()

    return conn, cursor


In [ ]:
def exit_connect_postgres(cursor, conn):
    cursor.close()
    conn.close()

    return None


In [ ]:
def create_db(database, new):
    # Conecta com o banco de dados
    conn, cursor = connect_db_postgres(database=database)

    # Consulta se já existe o banco de dados
    cursor.execute(
        "SELECT datname FROM pg_database WHERE datname = %s", (new,))
    row = cursor.fetchall()

    # Condição para criar um novo banco de dados
    if row:
        print("WARNING: Database {} Already Exists!".format(new))
    else:
        cursor.execute("CREATE DATABASE %s;", (AsIs(new),))
        print("Database {} Created!".format(new))

    # Fechando o cursor e a conexão
    exit_connect_postgres(cursor, conn)

    return None


In [ ]:
def create_db_schema(database, new):
    # Conecta com o banco de dados
    conn, cursor = connect_db_postgres(database=database)

    # Consulta se já existe o schema
    cursor.execute(
        "SELECT schema_name FROM information_schema.schemata WHERE schema_name = %s", (new,))
    row = cursor.fetchall()

    # Condição para criar um novo banco de dados
    if row:
        print("WARNING: Schema {} Already Exists!".format(new))
    else:
        cursor.execute("CREATE SCHEMA %s;", (AsIs(new),))
        print("SCHEMA {} Created!".format(new))

    # Fechando o cursor e a conexão
    exit_connect_postgres(cursor, conn)

    return None


In [ ]:
def create_table_relational_client(database, n_rows):
    # Conectando ao database
    conn, cursor = connect_db_postgres(database=database)

    # Consulta tabela se já existe
    cursor.execute(
        """SELECT EXISTS 
                (SELECT *
                    FROM information_schema.tables
                    WHERE table_schema = 'relacional' AND table_name = 'clients');
                    """)

    table_exists = cursor.fetchone()[0]

    # Condição para criar a tabela
    if table_exists:
        print("WARNING: Table Clients Already Exists!")
    else:
        cursor.execute("""
            CREATE SEQUENCE relacional.clientid;
            CREATE TABLE relacional.clients(
                clientid    int default nextval('relacional.clientid'::regclass) PRIMARY KEY,
                clientname  VARCHAR(300),
                email       VARCHAR(300),
                country     VARCHAR(255)
            );
        """)

    # Condição para inserir dados
    if table_exists:
        cursor.execute("""SELECT * 
                            FROM relacional.clients;
                       """)

        rows = cursor.fetchall()
        consulta_clientes = pd.DataFrame(
            rows, columns=['ClientID', 'ClientName',
                           'Email', 'Country'])

    else:
        print("Customer Table Created!")
        for data in client:
            cursor.execute("""
                INSERT INTO
                        relacional.clients (clientid, clientname, email, country)
                VALUES 
                        (%s, %s, %s, %s);
                """, data)
        cursor.execute("""SELECT *
                            FROM relacional.clients;
                       """)

        rows = cursor.fetchall()
        consulta_clientes = pd.DataFrame(
            rows, columns=['ClientID', 'ClientName',
                           'Email', 'Country'])

    # Fechando o cursor e a conexão
    exit_connect_postgres(cursor, conn)

    return consulta_clientes.head(n_rows)


In [ ]:
def create_table_relational_sales(database, n_rows):
    # Conectando ao database
    conn, cursor = connect_db_postgres(database=database)

    # Consulta tabela se já existe
    cursor.execute(
        """SELECT EXISTS
                (SELECT *
                    FROM information_schema.tables
                    WHERE table_schema = 'relacional' AND table_name = 'sales');
                    """)

    table_exists = cursor.fetchone()[0]

    # Condição para criar a tabela
    if table_exists:
        print("WARNING: Table Sales Already Exists!")
    else:
        cursor.execute("""
            CREATE SEQUENCE relacional.saleid;
            CREATE TABLE relacional.sales(
                         saleid     VARCHAR(15) default nextval('relacional.saleid'::regclass) PRIMARY KEY,
                         clientid   INT references relacional.clients (clientid),
                         invoice    INT,
                         date       DATE,
                         totalvalue DECIMAL(10,2)
            );
        """)

    # Condição para inserir dados
    if table_exists:
        cursor.execute("""SELECT *
                            FROM relacional.sales;""")

        rows = cursor.fetchall()
        consulta_vendas = pd.DataFrame(
            rows, columns=['saleid', 'clientid',
                           'invoice', 'date', 'totalvalue'])

    else:
        print("Sales Table Created!")
        for data in vendas:
            cursor.execute("""
                INSERT INTO 
                        relacional.sales (saleid, clientid, invoice, date, totalvalue)
                VALUES
                        (%s, %s, %s, %s, %s)
                """, data)

        cursor.execute("""SELECT *
                            FROM relacional.sales;
                       """)

        rows = cursor.fetchall()
        consulta_vendas = pd.DataFrame(
            rows, columns=['saleid', 'clientid',
                           'invoice', 'date', 'totalvalue'])

    # Fechando o cursor e a conexão
    exit_connect_postgres(cursor, conn)

    return consulta_vendas.head(n_rows)


In [ ]:
def create_table_relational_products(database, n_rows):
    # Conectando ao database
    conn, cursor = connect_db_postgres(database=database)

    # Consulta tabela se já existe
    cursor.execute(
        """SELECT EXISTS
                (SELECT *
                    FROM information_schema.tables
                    WHERE table_schema = 'relacional' AND table_name = 'products');
        """)

    table_exists = cursor.fetchone()[0]

    # Condição para criar a tabela
    if table_exists:
        print("WARNING: Table Products Already Exists!")
    else:
        cursor.execute("""
            CREATE SEQUENCE relacional.productid;
            CREATE TABLE relacional.products(
                         productid VARCHAR(15) default nextval('relacional.productid'::regclass) PRIMARY KEY,
                         product   VARCHAR(255),
                         price     DECIMAL(10,2)
            );
        """)

    # Condição para inserir dados
    if table_exists:
        cursor.execute("""SELECT *
                            FROM relacional.products;
                       """)
        rows = cursor.fetchall()
        consulta_produto = pd.DataFrame(
            rows, columns=['productid', 'product', 'price'])

    else:
        print("Products Table Created!")
        for data in produto:
            cursor.execute("""
                INSERT INTO
                        relacional.products (productid, product, price)
                VALUES 
                        (%s, %s, %s)
                """, data)

        cursor.execute("""SELECT *
                            FROM relacional.products;
                       """)
        rows = cursor.fetchall()
        consulta_produto = pd.DataFrame(
            rows, columns=['productid', 'product', 'price'])

    # Fechando o cursor e a conexão
    exit_connect_postgres(cursor, conn)

    return consulta_produto.head(n_rows)


In [ ]:
def create_table_relational_saleitems(database, n_rows):
    # Conectando ao database
    conn, cursor = connect_db_postgres(database=database)

    # Consulta tabela se já existe
    cursor.execute(
        """SELECT EXISTS
                (SELECT *
                    FROM information_schema.tables
                    WHERE table_schema = 'relacional' AND table_name = 'saleitems');
        """)

    table_exists = cursor.fetchone()[0]

    # Condição para criar a tabela
    if table_exists:
        print("WARNING: Table Saleitems Already Exists!")

    else:
        cursor.execute("""
            CREATE TABLE relacional.saleItems(
                        saleid varchar(15) REFERENCES relacional.sales ON DELETE CASCADE,
                        productid  VARCHAR(15) REFERENCES relacional.products ON DELETE RESTRICT,
                        invoice    INT,
                        quantity   INT,
                        price      DECIMAL(10,2),
                        totalvalue DECIMAL(10,2),
                        PRIMARY KEY (saleid, productid)
                       );
        """)

    # Condição para inserir dados
    if table_exists:
        # Consulta tabela
        cursor.execute("""SELECT *
                            FROM relacional.saleitems;
                       """)
        rows = cursor.fetchall()
        consulta_itensvenda = pd.DataFrame(
            rows, columns=['saleid', 'productid', 'invoice',
                           'quantity', 'price', 'totalvalue'])

    else:
        print("Saleitems Table Created!")
        for data in itensvenda:
            cursor.execute("""
                INSERT INTO
                           relacional.saleitems (saleid, productid, invoice, quantity, price, totalvalue)
                VALUES
                           (%s, %s, %s, %s, %s, %s)
                """, data)

        cursor.execute(
            "SELECT * FROM relacional.saleitems;")  # Consulta tabela
        rows = cursor.fetchall()
        consulta_itensvenda = pd.DataFrame(
            rows, columns=['saleid', 'productid', 'invoice',
                           'quantity', 'price', 'totalvalue'])

    # Fechando o cursor e a conexão
    exit_connect_postgres(cursor, conn)

    return consulta_itensvenda.head(n_rows)


In [ ]:
def create_table_dimensional_client(database, n_rows):
    # Conectando ao banco de dados
    conn, cursor = connect_db_postgres(database=database)

    # Consulta tabela
    cursor.execute(
        """SELECT EXISTS (
                        SELECT * 
                        FROM information_schema.tables 
                        WHERE table_schema = 'dimensional' AND table_name = 'dimclient'
                        );
                    """)
    table_exists = cursor.fetchone()[0]

    # Condição para criar a tabela
    if table_exists:
        print("WARNING: Table dimclient already exists!")
    else:
        cursor.execute("""
            CREATE SEQUENCE dimensional.keyclient;
            CREATE TABLE dimensional.dimclient(
                keyclient int default nextval('dimensional.keyclient'::regclass) PRIMARY KEY,
                client    VARCHAR(300),
                email     VARCHAR(300),
                country   VARCHAR(255)
            );
        """)

    # Condição para inserir dados
    if table_exists:
        cursor.execute("""
                       SELECT * 
                       FROM dimensional.dimclient;
                       """)
        consult_table = cursor.fetchall()
        consulta_sql = pd.DataFrame(
            consult_table, columns=[
                'keyclient',
                'client',
                'email',
                'country'
            ])
    else:
        print("dimclient Table Created!")
        cursor.execute("""
                       INSERT INTO 
                            dimensional.dimclient (keyclient, client, email, country)
                       SELECT 
                            clientid, clientname, email, country
                       FROM
                            relacional.clients;
                       """)

        # Consulta tabela
        cursor.execute("SELECT * FROM dimensional.dimclient;")
        consult_table = cursor.fetchall()
        consulta_sql = pd.DataFrame(
            consult_table, columns=[
                'keyclient',
                'client',
                'email',
                'country'
            ])

    # Fechando o cursor e a conexão
    exit_connect_postgres(cursor, conn)

    return consulta_sql.head(n_rows)


In [ ]:
def create_table_dimensional_product(database, n_rows):
    # Conectando ao banco de dados
    conn, cursor = connect_db_postgres(database=database)

    # Consulta tabela
    cursor.execute(
        """SELECT EXISTS (
                          SELECT *
                          FROM information_schema.tables
                          WHERE table_schema = 'dimensional' AND table_name = 'dimproduct'
                          );
                    """)
    table_exists = cursor.fetchone()[0]

    # Condição para criar a tabela
    if table_exists:
        print("WARNING: Table dimproduct already exists!")
    else:
        cursor.execute("""
            CREATE SEQUENCE dimensional.keyproduct;
            CREATE TABLE dimensional.dimproduct(
                keyproduct VARCHAR(15) default nextval('dimensional.keyproduct'::regclass) PRIMARY KEY,
                product    VARCHAR(255),
                price      DECIMAL(10,2)
            );
        """)

    # Condição para inserir dados
    if table_exists:
        cursor.execute("""
                       SELECT *
                       FROM dimensional.dimproduct;
                """)

        consult_table = cursor.fetchall()
        consulta_sql = pd.DataFrame(
            consult_table, columns=[
                'keyproduct',
                'product',
                'price'
            ])
    else:
        print("dimproduct Table Created!")
        cursor.execute("""
                       INSERT INTO
                            dimensional.dimproduct (keyproduct, product, price)
                       SELECT
                            productid, product, price
                       FROM
                            relacional.products;
                       """)

        cursor.execute("""
                       SELECT *
                       FROM dimensional.dimproduct;
                    """)

        consult_table = cursor.fetchall()
        consulta_sql = pd.DataFrame(
            consult_table, columns=[
                'keyproduct',
                'product',
                'price'
            ])

    # Fechando o cursor e a conexão
    exit_connect_postgres(cursor, conn)

    return consulta_sql.head(n_rows)


In [ ]:
def create_table_dimensional_sale(database, n_rows):
    # Conectando ao banco de dados
    conn, cursor = connect_db_postgres(database=database)

    # Consulta tabela
    cursor.execute(
        """SELECT EXISTS (
                          SELECT *
                          FROM information_schema.tables
                          WHERE table_schema = 'dimensional' AND table_name = 'fatsale'
                          );
                    """)
    table_exists = cursor.fetchone()[0]

    # Condição para criar a tabela
    if table_exists:
        print("WARNING: Table fatsale already exists!")
    else:
        cursor.execute("""
            CREATE SEQUENCE dimensional.keysale;
            CREATE TABLE dimensional.fatsale(
                keysale     VARCHAR(15) default nextval('dimensional.keysale'::regclass) PRIMARY KEY,
                keyclient   INT         REFERENCES  dimensional.dimclient   (keyclient),
                keyproduct  VARCHAR(15) REFERENCES  dimensional.dimproduct  (keyproduct),
                keytime     INT         REFERENCES  dimensional.dimtime     (keytime),
                invoice     INT,
                quantity    INT,
                price       DECIMAL(10,2),
                totalvalue  DECIMAL(10,2)
            );
        """)

    # Condição para inserir dados
    if table_exists:
        cursor.execute("""
                       SELECT *
                       FROM dimensional.fatsale;
                """)

        consult_table = cursor.fetchall()
        consulta_sql = pd.DataFrame(
            consult_table, columns=[
                'keysale',
                'keyclient',
                'keyproduct',
                'keytime',
                'invoice',
                'quantity',
                'price',
                'totalvalue'
            ])
    else:
        print("fatsale Table Created!")
        cursor.execute("""
                        INSERT INTO
                                dimensional.fatsale (keyclient, keyproduct, keytime, invoice, quantity, price, totalvalue)
                        SELECT
                            C.keyclient,
                            P.keyproduct,
                            T.keytime,
                            I.invoice,
                            I.quantity,
                            I.price,
                            I.totalvalue
                        FROM
                            relacional.sales V
                        INNER JOIN
                            relacional.saleitems I
                        ON
                            V.saleid = I.saleid
                        INNER JOIN
                            dimensional.dimclient C
                        ON
                            V.clientid = C.keyclient
                        INNER JOIN
                            dimensional.dimproduct P
                        ON
                            I.productid = P.keyproduct 
                        INNER JOIN
                            dimensional.dimtime T
                        ON
                            V.date = T.date;       
                 """)

        cursor.execute("""
                       SELECT *
                       FROM dimensional.fatsale;
                    """)

        consult_table = cursor.fetchall()
        consulta_sql = pd.DataFrame(
            consult_table, columns=[
                'keysale',
                'keyclient',
                'keyproduct',
                'keytime',
                'invoice',
                'quantity',
                'price',
                'totalvalue'
            ])

    # Fechando o cursor e a conexão
    exit_connect_postgres(cursor, conn)

    return consulta_sql.head(n_rows)


In [ ]:
def create_table_dimensional_time(database, n_rows):
    # Conectando ao banco de dados
    conn, cursor = connect_db_postgres(database=database)

    # Consulta tabela
    cursor.execute(
        """SELECT EXISTS (
                          SELECT *
                          FROM information_schema.tables
                          WHERE table_schema = 'dimensional' AND table_name = 'dimtime'
                          );
                    """)
    table_exists = cursor.fetchone()[0]

    # Condição para criar a tabela
    if table_exists:
        print("WARNING: Table dimtime already exists!")
    else:
        cursor.execute("""
            CREATE SEQUENCE dimensional.keytime;
            CREATE TABLE dimensional.dimtime(
                keytime INT default nextval('dimensional.keytime'::regclass) PRIMARY KEY,
                date    DATE,
                day     INT,
                month   INT,
                year    INT,
                weekday INT,
                quarter INT
            );
        """)

    # Condição para inserir dados
    if table_exists:
        cursor.execute("""
                       SELECT *
                       FROM dimensional.dimtime;
                       """)
        consult_table = cursor.fetchall()
        consulta_sql = pd.DataFrame(
            consult_table, columns=[
                'keytime',
                'date',
                'day',
                'month',
                'year',
                'weekday',
                'quarter'
            ])

    else:
        print("dimtime Table Created!")
        cursor.execute("""
                        INSERT INTO Dimensional.dimtime(date, day, month, year, weekday, quarter)
                        SELECT 
                            datum AS Data,
                                EXTRACT(DAY FROM datum)     AS Day,
                                EXTRACT(MONTH FROM datum)   AS Month,
                                EXTRACT(year FROM datum)    AS Year,
                                EXTRACT(dow FROM datum)     AS Weekday,
                                EXTRACT(quarter FROM datum) AS Qaurter                       
                        FROM
                            (SELECT '2009-01-01'::DATE+ SEQUENCE.DAY AS datum
                                FROM 
                                    GENERATE_SERIES (0,1461) AS SEQUENCE (DAY)
                                    GROUP BY 
                                            SEQUENCE.DAY) DQ
                        ORDER BY 1;
                """)

        cursor.execute("""
                       SELECT *
                       FROM dimensional.dimtime;
                       """)

        consult_table = cursor.fetchall()
        consulta_sql = pd.DataFrame(
            consult_table, columns=[
                'keytime',
                'date',
                'day',
                'month',
                'year',
                'weekday',
                'quarter'
            ])

    # Fechando o cursor e a conexão
    exit_connect_postgres(cursor, conn)

    return consulta_sql.head(n_rows)


In [ ]:
def create_view_table_sales(database, n_rows):
    # Conectando com o banco de dados
    conn, cursor = connect_db_postgres(database=database)

    # Consulta se a view
    cursor.execute("""
            SELECT EXISTS
                   (SELECT * 
                     FROM information_schema.views
                     WHERE table_schema = 'dimensional'
                     AND table_name = 'table_sales'
              );""")

    view_exists = cursor.fetchone()[0]

    # Condição para criar a view
    if view_exists:
        print("WARNING: View table_sales Already Axists!")
        cursor.execute("SELECT * FROM dimensional.table_sales;")
        consulta = cursor.fetchall()
        consulta = pd.DataFrame(consulta, columns=['client', 'email', 'country',
                                  'invoice', 'quantity', 'price', 'totalvalue',
                                  'product', 'date', 'day', 'month', 'year', 'quarter'])
    else:
        print("View table_sales Created Successfully!")
        cursor.execute("""
                CREATE VIEW 
                       dimensional.table_sales AS
                SELECT
                       dimensional.dimclient.client,
                       dimensional.dimclient.email,
                       dimensional.dimclient.country,
                       dimensional.fatsale.invoice,
                       dimensional.fatsale.quantity,
                       dimensional.fatsale.price,
                       dimensional.fatsale.totalvalue,
                       dimensional.dimproduct.product,
                       dimensional.dimtime.date,
                       dimensional.dimtime.day,
                       dimensional.dimtime.month,
                       dimensional.dimtime.year,
                       dimensional.dimtime.quarter       
       
                FROM
                       ((((dimensional.dimclient
                INNER JOIN
                       dimensional.fatsale
                ON
                       (dimensional.fatsale.keyclient = dimensional.dimclient.keyclient))
                INNER JOIN
                       dimensional.dimproduct
                ON
                       (dimensional.dimproduct.keyproduct = dimensional.fatsale.keyproduct))
                INNER JOIN
                       dimensional.dimtime
                ON
                       (dimensional.dimtime.keytime = dimensional.fatsale.keytime)))
        """)

        cursor.execute("SELECT * FROM dimensional.table_sales;")
        consulta = cursor.fetchall()
        consulta = pd.DataFrame(consulta, columns=['client', 'email', 'country',
                                  'invoice', 'quantity', 'price', 'totalvalue',
                                  'product', 'date', 'day', 'month', 'year', 'quarter'])

    # Fechando a conexão com o banco de dados
    exit_connect_postgres(cursor, conn)

    return consulta.head(n_rows)

In [ ]:
def consult_annual_billing(database):
    # Conectando ao database
    conn, cursor = connect_db_postgres(database=database)

    # Montando a query
    cursor.execute("""SELECT 
                     (SELECT SUM(totalValue) FROM dimensional.table_sales WHERE year = '2009') AS fat_2009,
                     (SELECT SUM(totalValue) FROM dimensional.table_sales WHERE year = '2010') AS fat_2010,
                     (SELECT SUM(totalValue) FROM dimensional.table_sales WHERE year = '2011') AS fat_2011;""")

    rows = cursor.fetchall()

    # Fechando conexão
    exit_connect_postgres(cursor, conn)

    # Formatando os valores da consulta
    fat_2009 = rows[0][0]
    fat_2010 = rows[0][1]
    fat_2011 = rows[0][2]

    # Dados
    df = pd.DataFrame({'Ano': [2009, 2010, 2011], 'Faturamento': [
        fat_2009, fat_2010, fat_2011]})

    # Criar rótulos personalizados com o ano e o valor do faturamento
    labels = [f"{ano}: $ {faturamento:,.2f}" for ano,
              faturamento in zip(df['Ano'], df['Faturamento'])]

    # Gráfico de Pizza
    f, ax = plt.subplots(figsize=(8, 8))
    ax.pie(df['Faturamento'], autopct='%1.1f%%', pctdistance=1.1, labeldistance=1.5,
           wedgeprops=dict(width=0.30), colors=['#4169E1', '#00BFFF', '#ADD8E6'])
    plt.legend(labels=labels, title='Ano e Faturamento',
               loc='center', prop={'size': 10})
    plt.title('Faturamento Anual')
    plt.show()

    return None


In [ ]:
def consult_biggest_customers(database):
    # Conectando ao database
    conn, cursor = connect_db_postgres(database=database)

    # Consultando os nomes dos clientes
    cursor.execute("""
                   SELECT client, SUM(totalvalue) AS total
                   FROM dimensional.table_sales
                   GROUP BY client
                   ORDER BY total DESC
                   LIMIT 5;""")

    rows = cursor.fetchall()

    # Fechando conexão
    exit_connect_postgres(cursor, conn)

    # Criando dataframe
    top_clientes = pd.DataFrame(
        rows, columns=['client', 'total'])

    # Gráfico de barras
    plt.figure(figsize=(7, 5))
    sns.barplot(x=top_clientes['total'],
                y=top_clientes['client'], orient='h', color='#4682B4')
    plt.ylabel('Clientes')
    plt.xlabel('Total Compras $')
    plt.title('Top 5 Maiores Clientes')
    plt.show()

    return None


In [ ]:
def consult_small_customers(database):
    # Conectando ao database
    conn, cursor = connect_db_postgres(database=database)

    # Consultando os nomes dos clientes
    cursor.execute("""
                   SELECT client, SUM(totalvalue) AS total
                   FROM dimensional.table_sales 
                   GROUP BY client
                   ORDER BY total ASC
                   LIMIT 5;""")

    rows = cursor.fetchall()

    # Fechando conexão
    exit_connect_postgres(cursor, conn)

    # Criando dataframe
    top_menores_clientes = pd.DataFrame(
        rows, columns=['client', 'total'])

    # Gráfico
    plt.figure(figsize=(7, 5))
    sns.barplot(x=top_menores_clientes['total'],
                y=top_menores_clientes['client'], orient='h', color='#4682B4')
    plt.ylabel('Clientes')
    plt.xlabel('Total Compras $')
    plt.title('Top 5 Menores Clientes')
    plt.show()

    return None


In [ ]:
def consult_best_selling_items(database):
    # Conectando ao database
    conn, cursor = connect_db_postgres(database=database)

    # Consultando os nomes dos itens
    cursor.execute("""
                   SELECT product, SUM(quantity) AS total
                   FROM dimensional.table_sales 
                   GROUP BY product
                   ORDER BY total DESC
                   LIMIT 5;""")

    rows = cursor.fetchall()

    # Fechando conexão
    exit_connect_postgres(cursor, conn)

    # Criando dataframe
    top_mercadorias = pd.DataFrame(
        rows, columns=['product', 'total'])

    # Gráfico de barras
    plt.figure(figsize=(8, 4))
    bar_plot = sns.barplot(
        x=top_mercadorias['product'],
        y=top_mercadorias['total'],
        orient='v', color='#4682B4')
    rotulos_quebrados = [label.replace(' ', '\n')
                         for label in top_mercadorias['product']]
    bar_plot.set_xticklabels(rotulos_quebrados, fontsize=9)
    plt.ylabel('Total Vendido')
    plt.xlabel('Produtos')
    plt.title('Top 5 Produtos Mais Vendidos')
    plt.show()

    return None


In [ ]:
def consult_less_sold_items(database):
    # Conectando ao database
    conn, cursor = connect_db_postgres(database=database)

    # Consultando os nomes dos itens
    cursor.execute("""
                   SELECT product, SUM(quantity) AS total
                   FROM dimensional.table_sales
                   GROUP BY product
                   ORDER BY total ASC
                   LIMIT 5;""")

    rows = cursor.fetchall()

    # Fechando conexão
    exit_connect_postgres(cursor, conn)

    # Criando dataframe
    top_menos_vendidos = pd.DataFrame(
        rows, columns=['product', 'total'])

    # Gráfico de barras
    plt.figure(figsize=(8, 4))
    bar_plot = sns.barplot(
        x=top_menos_vendidos['product'],
        y=top_menos_vendidos['total'],
        orient='v', color='#4682B4')
    rotulos_quebrados = [label.replace(' ', '\n')
                         for label in top_menos_vendidos['product']]
    bar_plot.set_xticklabels(rotulos_quebrados, fontsize=9)
    plt.ylabel('Total Vendido')
    plt.xlabel('Produtos')
    plt.title('Top 5 Produtos Mais Vendidos')
    plt.show()

    return None


## Carregando e tratando os dados

In [ ]:
# Acessando planilhas excel
arquivo = 'online_retail_II.xlsx'
data = open_data_execel(arquivo)

# Visualizando
data.head()


In [ ]:
# Dimesões da base de dados
data.shape


In [ ]:
# Verificando nulos
data.isnull().sum()


In [ ]:
# Verificando valores negativos e zerados base original
negative_values(data)


In [ ]:
# Limpeza e transformação da base
dataset = cleaning_transformation_data(data)


In [ ]:
# Verificando nulos
dataset.isnull().sum()


In [ ]:
# Valores únicos
dataset.nunique()


In [ ]:
# Visualizando base
dataset.head()


In [ ]:
# Tamanho da base
dataset.shape


## Construção de Tabelas

#### Clientes

In [ ]:
# Tabela Clientes
cliente = create_table_client(dataset)
cliente.head()


In [ ]:
# Dimensões
cliente.shape


In [ ]:
# Informações
cliente.info()


#### Vendas

In [ ]:
# Tabela Vendas
vendas = create_table_sales(dataset)
vendas.head()


In [ ]:
# Dimensões
vendas.shape


In [ ]:
# Informações do dataframe
vendas.info()


#### Produtos

In [ ]:
# Criando tabela produto
produto = create_table_product(dataset)
produto.head()


In [ ]:
# Visualizando o tamanho do dataframe
produto.shape


In [ ]:
# Informações do dataframe
produto.info()


#### Itens Vendidos

In [ ]:
# Criando tabela de itens vendidos
itensvenda = create_table_salesitems(dataset)
itensvenda.head()


In [ ]:
# Verificando tamanho do dataframe
itensvenda.shape


In [ ]:
# Informações do dataframe
itensvenda.info()


## Construção Database Relacional - PostgreSQL

### Construindo Database e Schema

In [ ]:
# Database
create_db(database='postgres', new='wgift')

# Schema
create_db_schema(database='wgift', new='relacional')


### Construção de Tabelas e Inserção de Dados

In [ ]:
# Criando tabela cliente no databse relacional wgift
create_table_relational_client(database='wgift', n_rows=10)


**Consulta tabela clientes no Postegres:**

<img src='table-clients.jpg'>

In [ ]:
# Criando tabela vendas no databse relacional wgift
create_table_relational_sales(database='wgift', n_rows=10)


**Consulta tabela vendas no Postegres:**

<img src='table-sales.jpg'>

In [ ]:
# Criando tabela produtos no databse relacional wgift
create_table_relational_products(database='wgift', n_rows=10)


**Consulta tabela produtos no Postegres:**

<img src='table-products.jpg'>

In [ ]:
# Criando tabela itens vendidos no databse relacional wgift
create_table_relational_saleitems(database='wgift', n_rows=10)


**Consulta tabela itensvenda no Postegres:**

<img src='table-saleitems.jpg'>

## Construção Data WhereHouse - PostgreSQL

In [ ]:
# Criando schema dimensional no banco de dados wgift
create_db_schema(database='wgift', new='dimensional')


In [ ]:
# Criando tabela dimensional dimclient e inserindo dados da tabela relacional clients
create_table_dimensional_client(database='wgift', n_rows=10)


In [ ]:
# Criando tabela dimensional dimproduct e inserindo dados da tabela relacional products
create_table_dimensional_product(database='wgift', n_rows=10)


In [ ]:
# Criando tabela dimensional dimtime e inserindo dados
create_table_dimensional_time(database='wgift', n_rows=10)


In [ ]:
# Criando tabela fato fatsale e inserindo dados
create_table_dimensional_sale(database='wgift', n_rows=10)


## Consultas SQL - Database Relacional

##### View

In [ ]:
# Criando view table_sales para análise e montagem de dashboard
create_view_table_sales(database='wgift', n_rows=10)

#### Consultas

Qual o Faturamento Anual da Empresa?

In [ ]:
consult_annual_billing(database='wgift')


Quais os 5 Maiores Clientes?

In [ ]:
consult_biggest_customers(database='wgift')


Quais os 5 Menores Clientes?

In [ ]:
consult_small_customers(database='wgift')


Quais os 5 Itens mais Vendidos?

In [ ]:
consult_best_selling_items(database='wgift')


Quais os 5 Itens menos Vendidos?

In [ ]:
consult_less_sold_items(database='wgift')


# Fim